In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
# 경고 메시지 숨기기
warnings.filterwarnings("ignore")

In [39]:
df = pd.read_csv("./data/data_total.csv")

### 날짜 타입 변경 및 년/월/일 분리

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856566 entries, 0 to 856565
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              856566 non-null  int64  
 1   msno                    856566 non-null  int64  
 2   city                    856566 non-null  int64  
 3   bd                      856566 non-null  int64  
 4   gender                  368210 non-null  object 
 5   registered_via          856566 non-null  int64  
 6   registration_init_time  856566 non-null  object 
 7   is_churn                856566 non-null  int64  
 8   is_back                 856566 non-null  int64  
 9   payment_plan_sum        856566 non-null  int64  
 10  plan_list_price         856566 non-null  int64  
 11  actual_amount_paid      856566 non-null  int64  
 12  discount_rate           856566 non-null  float64
 13  is_auto_renew           856566 non-null  float64
 14  membership_expire_da

In [44]:
def int_to_date(df):
    # membership_expire_date와 registration_init_time은 'YYYY-MM-DD' 형식으로 되어 있으므로, 그에 맞춰 변환
    df['membership_expire_date'] = pd.to_datetime(df['membership_expire_date'], errors='coerce')
    df['registration_init_time'] = pd.to_datetime(df['registration_init_time'], errors='coerce')
    
    # log_start와 log_end는 'YYYYMMDD' 형식이므로, 그에 맞춰 변환
    df['log_start'] = pd.to_datetime(df['log_start'].astype(str), format='%Y%m%d', errors='coerce')
    df['log_end'] = pd.to_datetime(df['log_end'].astype(str), format='%Y%m%d', errors='coerce')

    return df

# 함수 실행
df = int_to_date(df)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856566 entries, 0 to 856565
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Unnamed: 0              856566 non-null  int64         
 1   msno                    856566 non-null  int64         
 2   city                    856566 non-null  int64         
 3   bd                      856566 non-null  int64         
 4   gender                  368210 non-null  object        
 5   registered_via          856566 non-null  int64         
 6   registration_init_time  856566 non-null  datetime64[ns]
 7   is_churn                856566 non-null  int64         
 8   is_back                 856566 non-null  int64         
 9   payment_plan_sum        856566 non-null  int64         
 10  plan_list_price         856566 non-null  int64         
 11  actual_amount_paid      856566 non-null  int64         
 12  discount_rate           856566

* 년,월,일 분리

In [47]:

def split_date_columns(df, column_name):
    # 컬럼을 datetime 형식으로 변환 (변환 실패시 NaT로 처리)
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    
    # 연도, 월, 일 컬럼 추가
    df[f'{column_name}_year'] = df[column_name].dt.year
    df[f'{column_name}_month'] = df[column_name].dt.month
    df[f'{column_name}_day'] = df[column_name].dt.day
    
    # 원본 컬럼 삭제
    df = df.drop(columns=[column_name])
    
    return df

In [48]:
df = split_date_columns(df, 'registration_init_time')
df = split_date_columns(df, 'membership_expire_date')
df = split_date_columns(df, 'log_end')
df = split_date_columns(df, 'log_start')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856566 entries, 0 to 856565
Data columns (total 34 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    856566 non-null  int64  
 1   msno                          856566 non-null  int64  
 2   city                          856566 non-null  int64  
 3   bd                            856566 non-null  int64  
 4   gender                        368210 non-null  object 
 5   registered_via                856566 non-null  int64  
 6   is_churn                      856566 non-null  int64  
 7   is_back                       856566 non-null  int64  
 8   payment_plan_sum              856566 non-null  int64  
 9   plan_list_price               856566 non-null  int64  
 10  actual_amount_paid            856566 non-null  int64  
 11  discount_rate                 856566 non-null  float64
 12  is_auto_renew                 856566 non-nul

### 성별 인코딩

In [52]:
def encode_gender(df):
    # 성별 인코딩: M -> 1, F -> 0, nan -> -1 (또는 원하는 값으로 변경)
    df['gender_encoded'] = df['gender'].map({'M': 1, 'F': 0})
    
    # nan 값을 -1로 처리 (혹은 다른 값으로 처리)
    df['gender_encoded'] = df['gender_encoded'].fillna(-1)  # nan을 -1로 처리
    df = df.drop(columns=["gender"])
    
    return df

In [53]:
df = encode_gender(df)

### 정규화

### 데이터 저장

In [54]:
df.to_csv("./data/data_total_split_datetime")